# Human Entity in Al Qur'an with NER & Rule Based

Import Modules

In [ ]:
import pandas as pd

## 1. Buckwalter
Mengubah form latin dalam dataset qorpus menjadi arab

In [ ]:
# Install module
# pip install lang-trans

In [ ]:
# Import package
from lang_trans.arabic import buckwalter

In [ ]:
# Import Dataset
quranicDF = pd.read_excel('./Dataset/quranic-corpus-morphology-0.4.xlsx', index_col=0)

In [ ]:
# Show first 5 row data
quranicDF.head()

In [ ]:
# First Buckwaltering
arab = []

for i in range (len(quranicDF)):
    if (pd.isna(quranicDF.iloc[i,0])):
        arab.append("")
    else:
        arab.append(buckwalter.untransliterate(quranicDF.iloc[i,0]))
    (str(i) + ' ' +arab[i])

#### Second Bucwaltering
Menghilangkan 3 char yang aneh dan mereplace 1 char lalu bucwaltering semuanya ke arab

In [ ]:
# Import package regular expression
import re

arab = []

for i in range (len(quranicDF)):
    
    if (pd.isna(quranicDF.iloc[i,0])):
        arab.append("")
    else:
    
        quranicDF.iloc[i,0] = re.sub(r'^aA', 'A`', quranicDF.iloc[i,0]) #belum paham parameter string/df.iloc[i,0] #Done

        if quranicDF.iloc[i,0] == 'a':
            arab.append(quranicDF.iloc[i,0].replace(r'^a$', '>a')) #belum paham $ nya #Done
        else:
            arab.append(buckwalter.untransliterate(quranicDF.iloc[i,0]))
            
        arab[i] = arab[i].replace('^', '')
        arab[i] = arab[i].replace('@', '')
        arab[i] = arab[i].replace('[', '')
    str(i) + ' ' + arab[i]

In [ ]:
# insert data to variable
quranicDF['ARAB'] = arab
arab

In [ ]:
# export data
quranicDF.to_excel("./Output/quranic-corpus-morphology-0.4-with-arab.xlsx")
# quranic_df.to_csv("./Output/quranic-corpus-morphology-0.4-with-arab.csv")

## 2. Get Human Entity Rule

In [ ]:
# Import dataset
entityDF = pd.read_excel('./Dataset/human-entity-juz-3.xlsx')
pd.set_option('display.max_rows', 999999)

In [ ]:
# Import dataset
entityDF = pd.read_excel('./Dataset/human-entity-10-juz.xlsx')
pd.set_option('display.max_rows', 999999)

In [ ]:
def findHumanEntityPatterns(file):
    
    tempPatterns = [[]]
    finalPatterns = []
    currentTag = 0
    opentagCount = 0
    df = pd.DataFrame({'pattern': []})
    
    for i in range(len(file)):

        closetags = [char for char in str(file.at[i, "closetag"])]
        opentags = [char for char in str(file.at[i, "opentag"])]

        for j in range(len(opentags)):
            if opentags[j] == '(':
                opentagCount += 1

        if opentagCount > 0:
            tempPatterns[currentTag].append(file.at[i, "tag"])

        for k in range(len(closetags)):
            if closetags[k] == ')':
                if opentagCount == 1:
                    toInput = (str(tempPatterns[currentTag]))
                    df = df.append({'pattern' : toInput}, ignore_index=True)
                    finalPatterns.append(tempPatterns[currentTag])
                    currentTag += 1
                    tempPatterns.append([])
                opentagCount -= 1
                
    if tempPatterns[len(tempPatterns)-1] == []:
        del tempPatterns[len(tempPatterns)-1]

    # Ini apaan coba
    tempPatterns = set(tuple(element) for element in tempPatterns)
    tempPatterns = [list(t) for t in set(tuple(element) for element in tempPatterns)]    
        
#     return tempPatterns
    return df

In [ ]:
patterns = findHumanEntityPatterns(entityDF)

Use return df for good looking data

In [ ]:
print (len(patterns))
patterns

Remove Duplicate

In [ ]:
patterns = patterns.drop_duplicates(subset=None, keep='first', inplace=False)
print(len(patterns))
patterns

## 3. Set Human Entity Rule

In [ ]:
# Import data
quranicArab = pd.read_excel('./Output/quranic-corpus-morphology-0.4-with-arab.xlsx', encoding='utf8')
pd.set_option('display.max_rows', 999999) # Untuk menampilkan semua data sekaligus

In [ ]:
# Prepare data
def convertAndGroupToArray(data):
    
    ayats = [[]]

    for index, row in data.iterrows():
        location = row['LOCATION']
        ayat = location.split(':')[1]
        data.at[index, 'LOCATION'] = ayat

    length = len(data)
    i = 0
    data.at[0, 'LOCATION'] = 0
    
    for index, row in data.iterrows():
        
        # tokenizing location
        if index < length-1:
            currentLocation = row['LOCATION']
            nextLocation = data.at[index+1, 'LOCATION']

            if index == length-2 or currentLocation != nextLocation:
                i = i+1
                ayats.append([])
            
            data.at[index, 'LOCATION'] = i
            
            words = {
                'OPEN TAG' : '',
                'AYAT' : int(row['LOCATION']),
                'ARAB' : row['ARAB'],
                'TAG' : row['TAG'],
                'CLOSE TAG' : ''
            }
            
            ayats[i].append(words)
    
    return ayats
     
ayats = convertAndGroupToArray(quranicArab[:5000])

In [ ]:
# Rule based logic
def findHumanEntity(ayats):
    
    ayatIndex = 0
    entityIndex = 0
    for ayat in ayats:
        for pattern in patterns:
            patternLength = len(pattern)
            for wordIndex in range(len(ayat) - patternLength):
                # Get set of words based on the pattern length
                wordTagsSet = []
                for wordOfSetIndex in range((patternLength)):
                    wordTagsSet.append(ayat[wordIndex+wordOfSetIndex]['TAG'])
                if str(wordTagsSet) == str(pattern):
                    ayat[wordIndex]['OPEN TAG'] += str(entityIndex) + '('
                    ayat[wordIndex+patternLength]['CLOSE TAG'] += str(entityIndex) + ')'
                    entityIndex += 1
        ayatIndex += 1
                
findHumanEntity(ayats)

In [ ]:
ayatDF = pd.DataFrame({'AYAT': [], 'OPEN TAG': [], 'ARAB': [], 'TAG': [], 'CLOSE TAG': []});

ayatCollections = []
openTagCollections = []
arabCollections = []
tagCollections = []
closeTagCollections = []

for ayat in ayats:
    for word in ayat:
        ayatDF = ayatDF.append({'AYAT': word['AYAT'], 'OPEN TAG': word['OPEN TAG'], 'ARAB': word['ARAB'], 'TAG': word['TAG'], 'CLOSE TAG': word['CLOSE TAG']}, ignore_index=True)
        
ayatDF

## 4. Placing Space

In [ ]:
dataForEntity = pd.read_excel('./Output/quranic-corpus-morphology-0.4-with-arab.xlsx')
dataOfStandardizationResource = pd.read_excel('./Dataset/wordbywordFromCorpus.xlsx')

def strip_arabic_vowels(line_with_vowels):
    diacritics = [u'\u064e',  # fatha, short a
                  u'\u064b',  # double fatha
                  u'\u0650',  # kasra, short i
                  u'\u064d',  # double kasra
                  u'\u064f',  # damma, short u
                  u'\u064c',  # double damma
                  u'\u0652',  # sukkun, nothing
                  u'\u0651',  # shedda, double
                  u'\u0670',
                  u'\u0671',
                  u'\u0653',
                  u'\u08EA',
                  u'\uFBB2',
                  u'\u0660',
                  u'\u06DF',
                  u'\u06E2',
                  u'\u06E5',
                  ',',
                  '.',
                  u'\u06E7',
                  u'\u06E6',
                  u'\u0654',
                  u'\u06ED',
                  '#',
                  '[',
                  ']',
                  ':',
                  u'\u06DC',
                  '"',
                  u'\u06e0',
                  u'\u06e8'
                 ]
    
    line_without_vowels = ''
    for char in line_with_vowels:
        if char not in diacritics:
            line_without_vowels += char
    
    return line_without_vowels
    
dosrIndex = 0

import re

for i in range(len(dataForEntity)):
    query = strip_arabic_vowels(str(dataForEntity.at[i, 'ARAB']))
    nextQuery = strip_arabic_vowels(str(dataForEntity.at[i+1, 'ARAB']))
    source = strip_arabic_vowels(str(dataOfStandardizationResource.at[dosrIndex, 'Data']))
    
    print(str(i) + ' | ' + query + ' | ' + source)
    
    exist = re.search(query + "$", source)
    queryLocation = -1
    
    nextQueryExist = re.search(nextQuery + "$", source)

    if exist is not None:
        queryLocation = exist.span()[0]
    
    if nextQueryExist is not None:
        if nextQueryExist.span()[0] == 0:
            nextQueryExist = False
            
    queryShouldLastIndex = len(source) - len(query)
    
    if exist and queryShouldLastIndex == queryLocation:
        dataForEntity.at[i, 'SPACE AFTER'] = 'TRUE'
        print("Last")
        dosrIndex += 1
    else:
        print("Not last") 
            
# pd.set_option("display.max_rows", 99999999)
dataForEntity

berhenti di 74359

In [ ]:
dataForEntity.to_excel("./Output/quranic-corpus-morphology-0.4-with-arab-and-space.xlsx")

## 5. Insert Dataset & Rule to Mongo

In [ ]:
# import package
import pymongo

# connecting to mongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# create database
mydb = myclient["TA"]

# create collection
quran = mydb["quran"]

# import dataset
data = pd.read_excel("./Output/quranic-corpus-morphology-0.4-with-arab-and-space.xlsx")

Split location dataset

In [ ]:
for i in range(len(data)):
    location = data.at[i, "LOCATION"]
    location = location.split(':')
    no_surat = location[0][1:] # belum paham
    ayat = location[1]
    word = location[2]
    
    data.at[i, 'SURAH_NUMBER'] = no_surat
    data.at[i, 'AYAT_NUMBER'] = ayat
    data.at[i, 'WORD_NUMBER'] = word

In [ ]:
# cek data
data.iloc[5]

Insert dataset

In [ ]:
# insert to mongo
quran.insert_many(data.to_dict('records'))

In [ ]:
x = quran.find({"SURAH_NUMBER": "67"})

Insert patterns

In [ ]:
# patterns = [['REL', 'V'], ['PN', 'N', 'PN'], ['REL', 'P', 'N', 'PRON'], ['REL', 'V', 'PRON'], ['N', 'PRON'], ['N'], ['COND', 'V', 'P', 'DET', 'N', 'CONJ', 'V', 'P', 'PN'], ['REL', 'V', 'PN'], ['PN'], ['N', 'DET', 'ADJ'], ['REL', 'V', 'P', 'N'], ['REL', 'V', 'PRON', 'N', 'PRON', 'P', 'N', 'PN'], ['REL', 'V', 'N', 'PRON'], ['ADJ'], ['REL', 'V', 'PRON', 'N', 'PRON', 'N', 'N', 'PN'], ['COND', 'V', 'DET', 'N'], ['N', 'DET', 'N'], ['DET', 'N'], ['REL', 'V', 'PRON', 'P', 'N', 'PN'], ['REL', 'V', 'PRON', 'N', 'PRON', 'P', 'DET', 'N', 'CONJ', 'DET', 'N'], ['REL', 'V', 'PRON', 'DET', 'N'], ['COND', 'V', 'PRON'], ['COND', 'V'], ['N', 'DET', 'N', 'PRON'], ['REL', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'DET', 'N', 'CONJ', 'V', 'PRON', 'DET', 'N', 'CONJ', 'V', 'PRON', 'DET', 'N'], ['REL', 'P', 'PRON', 'DET', 'N'], ['COND', 'P', 'N', 'PRON', 'N'], ['N', 'PN'], ['N', 'CONJ', 'DET', 'N'], ['REL', 'V', 'PRON', 'N', 'PRON', 'ACC', 'PRON', 'V', 'PRON', 'REM', 'V', 'P', 'PRON', 'N', 'PRON', 'CONJ', 'V', 'PRON', 'N', 'DET', 'N', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'N', 'P', 'DET', 'N', 'V', 'PN', 'ACC', 'PRON', 'NEG', 'N', 'EXP', 'PRON', 'CONJ', 'DET', 'N', 'CONJ', 'N', 'DET', 'N', 'N', 'P', 'DET', 'N', 'NEG', 'N', 'EXP', 'PRON', 'DET', 'N', 'DET', 'ADJ', 'ACC', 'DET', 'N', 'LOC', 'PN', 'DET', 'PN', 'REM', 'NEG', 'V', 'REL', 'V', 'PRON', 'DET', 'N', 'RES', 'P', 'N', 'REL', 'V', 'PRON', 'DET', 'N', 'N', 'LOC', 'PRON', 'REM', 'COND', 'V', 'P', 'N', 'PN', 'RSLT', 'ACC', 'PN', 'N', 'DET', 'N', 'REM', 'COND', 'V', 'PRON', 'PRON', 'RSLT', 'V', 'V', 'PRON', 'N', 'PRON', 'P', 'PN', 'CONJ', 'REL', 'V', 'PRON', 'CONJ', 'V', 'P', 'REL', 'V', 'PRON', 'DET', 'N', 'CONJ', 'DET', 'N', 'INTG', 'V', 'PRON', 'REM', 'COND', 'V', 'PRON', 'RSLT', 'CERT', 'V', 'PRON', 'CONJ', 'COND', 'V', 'PRON', 'RSLT', 'ACC', 'PREV', 'P', 'PRON', 'DET', 'N', 'REM', 'PN', 'N', 'P', 'DET', 'N', 'ACC', 'REL', 'V', 'PRON', 'P', 'N', 'PN', 'CONJ', 'V', 'PRON', 'DET', 'N', 'P', 'N', 'N', 'CONJ', 'V', 'PRON', 'REL', 'V', 'PRON', 'P', 'DET', 'N', 'P', 'DET', 'N', 'REM', 'V', 'PRON', 'P', 'N', 'ADJ', 'DEM', 'REL', 'V', 'N', 'PRON', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'REM', 'NEG', 'P', 'PRON', 'P', 'N', 'INTG', 'NEG', 'V', 'P', 'REL', 'V', 'PRON', 'N', 'P', 'DET', 'N', 'V', 'PRON', 'P', 'N', 'PN', 'PRP', 'V', 'LOC', 'PRON', 'CONJ', 'V', 'N', 'P', 'PRON', 'CIRC', 'PRON', 'N', 'DEM', 'P', 'ACC', 'PRON', 'V', 'PRON', 'NEG', 'V', 'PRON', 'DET', 'N', 'RES', 'T', 'ADJ', 'REM', 'V', 'PRON', 'P', 'N', 'PRON', 'REL', 'V', 'PRON', 'V', 'PRON', 'REM', 'INTG', 'T', 'V', 'PRON', 'PRON', 'P', 'N', 'NEG', 'N', 'P', 'PRON', 'CONJ', 'V', 'N', 'N', 'REL', 'V', 'CIRC', 'PRON', 'NEG', 'V', 'PRON', 'V', 'PN', 'VOC', 'N', 'DET', 'N', 'V', 'DET', 'N', 'REL', 'V', 'CONJ', 'V', 'DET', 'N', 'P', 'REL', 'V', 'CONJ', 'V', 'REL', 'V', 'CONJ', 'V', 'REL', 'V', 'P', 'N', 'PRON', 'DET', 'N', 'ACC', 'PRON', 'P', 'N', 'N', 'N', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'V', 'REL', 'V', 'P', 'N', 'N', 'PRO', 'V', 'DET', 'N', 'DET', 'N', 'N', 'P', 'N', 'DET', 'N', 'REM', 'COND', 'V', 'DEM', 'RSLT', 'V', 'P', 'PN', 'P', 'N', 'RES', 'SUB', 'V', 'PRON', 'P', 'PRON', 'N', 'REM', 'V', 'PRON', 'PN', 'N', 'PRON', 'REM', 'P', 'PN', 'DET', 'N', 'V', 'COND', 'V', 'PRON', 'REL', 'P', 'N', 'PRON', 'CONJ', 'V', 'PRON', 'PRON', 'V', 'PRON', 'PN', 'REM', 'V', 'REL', 'P', 'DET', 'N', 'CONJ', 'REL', 'P', 'DET', 'N', 'REM', 'PN', 'P', 'N', 'N', 'N', 'T', 'V', 'N', 'N', 'REL', 'V', 'P', 'N', 'N', 'CONJ', 'REL', 'V', 'P', 'N', 'V', 'SUB', 'ACC', 'LOC', 'PRON', 'CONJ', 'LOC', 'PRON', 'N', 'ADJ', 'REM', 'V', 'PRON', 'PN', 'N', 'PRON', 'REM', 'PN', 'N', 'P', 'DET', 'N', 'V', 'COND', 'V', 'PRON', 'V', 'PRON', 'PN', 'RSLT', 'V', 'PRON', 'PRON', 'V', 'PRON', 'PN', 'CONJ', 'V', 'P', 'PRON', 'N', 'PRON', 'REM', 'PN', 'N', 'ADJ', 'V', 'V', 'PRON', 'PN', 'CONJ', 'DET', 'N', 'REM', 'COND', 'V', 'PRON', 'RSLT', 'ACC', 'PN', 'NEG', 'V', 'DET', 'N', 'ACC', 'PN', 'V', 'PN', 'CONJ', 'PN', 'CONJ', 'N', 'PN', 'CONJ', 'N', 'PN', 'P', 'DET', 'N', 'N', 'N', 'PRON', 'P', 'N', 'REM', 'PN', 'N', 'ADJ', 'T', 'V', 'N', 'PN', 'N', 'PRON', 'ACC', 'PRON', 'V', 'PRON', 'P', 'PRON', 'REL', 'P', 'N', 'PRON', 'N', 'REM', 'V', 'P', 'PRON', 'ACC', 'PRON', 'PRON', 'DET', 'N', 'DET', 'ADJ', 'REM', 'T', 'V', 'PRON', 'V', 'N', 'PRON', 'ACC', 'PRON', 'V', 'PRON', 'PRON', 'N', 'REM', 'PN', 'N', 'P', 'REL', 'V', 'REM', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'ACC', 'PRON', 'V', 'PRON', 'PRON', 'PN', 'CONJ', 'ACC', 'PRON', 'V', 'PRON', 'P', 'PRON', 'CONJ', 'N', 'PRON', 'P', 'DET', 'PN', 'DET', 'ADJ', 'REM', 'V', 'PRON', 'N', 'PRON', 'P', 'N', 'ADJ', 'CONJ', 'V', 'PRON', 'N', 'ADJ', 'CONJ', 'V', 'PRON', 'PN', 'T', 'V', 'P', 'PRON', 'PN', 'DET', 'N', 'V', 'LOC', 'PRON', 'N', 'V', 'VOC', 'PN', 'INTG', 'P', 'PRON', 'DEM', 'V', 'PRON', 'P', 'N', 'PN', 'ACC', 'PN', 'V', 'REL', 'V', 'P', 'N', 'N', 'T', 'V', 'PN', 'N', 'PRON', 'V', 'N', 'PRON', 'V', 'P', 'PRON', 'P', 'N', 'PRON', 'N', 'ADJ', 'ACC', 'PRON', 'N', 'DET', 'N', 'REM', 'V', 'PRON', 'DET', 'N', 'CIRC', 'PRON', 'N', 'V', 'P', 'DET', 'N', 'ACC', 'PN', 'V', 'PRON', 'P', 'PN', 'N', 'P', 'N', 'P', 'PN', 'CONJ', 'N', 'CONJ', 'N', 'CONJ', 'N', 'P', 'DET', 'N', 'V', 'N', 'PRON', 'INTG', 'V', 'P', 'PRON', 'N', 'CIRC', 'CERT', 'V', 'PRON', 'DET', 'N', 'CONJ', 'N', 'PRON', 'N', 'V', 'P', 'DEM', 'PN', 'V', 'REL', 'V', 'V', 'N', 'PRON', 'V', 'P', 'PRON', 'N', 'V', 'N', 'PRON', 'SUB', 'NEG', 'V', 'DET', 'N', 'T', 'N', 'EXP', 'N', 'CONJ', 'V', 'N', 'PRON', 'ADJ', 'CONJ', 'V', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'T', 'V', 'DET', 'N', 'VOC', 'PN', 'ACC', 'PN', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'P', 'N', 'DET', 'N', 'VOC', 'PN', 'V', 'PRON', 'P', 'N', 'PRON', 'CONJ', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'LOC', 'DET', 'N', 'DEM', 'P', 'N', 'DET', 'N', 'V', 'PRON', 'P', 'PRON', 'REM', 'NEG', 'V', 'PRON', 'LOC', 'PRON', 'T', 'V', 'PRON', 'N', 'PRON', 'INTG', 'PRON', 'V', 'PN', 'CONJ', 'NEG', 'V', 'PRON', 'LOC', 'PRON', 'T', 'V', 'PRON', 'T', 'V', 'DET', 'N', 'VOC', 'PN', 'ACC', 'PN', 'V', 'PRON', 'P', 'N', 'P', 'PRON', 'N', 'PRON', 'DET', 'PN', 'PN', 'N', 'PN', 'N', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'P', 'DET', 'N', 'CONJ', 'V', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'N', 'CONJ', 'P', 'DET', 'N', 'V', 'N', 'PRON', 'INTG', 'V', 'P', 'PRON', 'N', 'CIRC', 'NEG', 'V', 'PRON', 'N', 'V', 'P', 'DEM', 'PN', 'V', 'REL', 'V', 'T', 'V', 'N', 'RSLT', 'ACC', 'PREV', 'V', 'P', 'PRON', 'V', 'CONJ', 'V', 'REM', 'V', 'PRON', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'PN', 'CONJ', 'DET', 'PN', 'CONJ', 'N', 'P', 'N', 'PN', 'ACC', 'PRON', 'CERT', 'V', 'PRON', 'PRON', 'P', 'N', 'P', 'N', 'PRON', 'ACC', 'PRON', 'V', 'P', 'PRON', 'P', 'DET', 'N', 'P', 'N', 'DET', 'N', 'CONJ', 'V', 'P', 'PRON', 'CONJ', 'V', 'N', 'P', 'N', 'PN', 'CONJ', 'V', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'V', 'DET', 'N', 'P', 'N', 'PN', 'CONJ', 'V', 'PRON', 'P', 'REL', 'V', 'PRON', 'CONJ', 'REL', 'V', 'PRON', 'P', 'N', 'PRON', 'ACC', 'P', 'DEM', 'EMPH', 'N', 'P', 'PRON', 'COND', 'V', 'PRON', 'N', 'CONJ', 'N', 'P', 'REL', 'LOC', 'N', 'PRON', 'P', 'DET', 'PN', 'CONJ', 'PRP', 'V', 'P', 'PRON', 'N', 'REL', 'V', 'P', 'PRON', 'REM', 'V', 'PRON', 'PRON', 'P', 'N', 'P', 'N', 'PRON', 'REM', 'V', 'PRON', 'PN', 'CONJ', 'V', 'PRON', 'PRON', 'ACC', 'PN', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'REM', 'V', 'PRON', 'PRON', 'DEM', 'N', 'ADJ', 'REM', 'T', 'V', 'PN', 'P', 'PRON', 'DET', 'N', 'V', 'INTG', 'N', 'PRON', 'P', 'PN', 'V', 'DET', 'N', 'PRON', 'N', 'PN', 'V', 'PRON', 'P', 'PN', 'REM', 'V', 'P', 'ACC', 'PRON', 'N', 'N', 'PRON', 'V', 'PRON', 'P', 'REL', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'DET', 'N', 'REM', 'V', 'PRON', 'LOC', 'DET', 'N', 'REM', 'V', 'PRON', 'CONJ', 'V', 'PN', 'REM', 'PN', 'N', 'DET', 'N', 'T', 'V', 'PN', 'VOC', 'PN', 'ACC', 'PRON', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'P', 'PRON', 'CONJ', 'N', 'PRON', 'P', 'REL', 'V', 'PRON', 'CONJ', 'N', 'REL', 'V', 'PRON', 'PRON', 'LOC', 'REL', 'V', 'PRON', 'P', 'N', 'DET', 'N', 'CONJ', 'P', 'PRON', 'N', 'PRON', 'CONJ', 'V', 'LOC', 'PRON', 'P', 'REL', 'V', 'PRON', 'P', 'PRON', 'V', 'PRON', 'REM', 'EXL', 'COND', 'V', 'PRON', 'RSLT', 'V', 'PRON', 'N', 'ADJ', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'REM', 'NEG', 'P', 'PRON', 'P', 'N', 'CONJ', 'EXL', 'COND', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'DET', 'N', 'RSLT', 'V', 'PRON', 'N', 'PRON', 'REM', 'PN', 'NEG', 'V', 'DET', 'N', 'DEM', 'V', 'PRON', 'P', 'PRON', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'DET', 'ADJ', 'ACC', 'N', 'PN', 'LOC', 'PN', 'P', 'N', 'PN', 'V', 'PRON', 'P', 'N', 'CONJ', 'V', 'P', 'PRON', 'V', 'REM', 'V', 'DET', 'N', 'P', 'N', 'PRON', 'REM', 'PRO', 'V', 'P', 'DET', 'N', 'REM', 'COND', 'V', 'PRON', 'P', 'PRON', 'P', 'N', 'REL', 'V', 'PRON', 'P', 'DET', 'N', 'RSLT', 'V', 'V', 'PRON', 'V', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'CONJ', 'N', 'PRON', 'CONJ', 'V', 'CONJ', 'V', 'N', 'PN', 'P', 'DET', 'N', 'ACC', 'DEM', 'EMPH', 'PRON', 'DET', 'N', 'DET', 'ADJ', 'REM', 'NEG', 'P', 'N', 'RES', 'PN', 'REM', 'ACC', 'PN', 'EMPH', 'PRON', 'DET', 'N', 'DET', 'ADJ', 'REM', 'COND', 'V', 'PRON', 'RSLT', 'ACC', 'PN', 'N', 'P', 'DET', 'N', 'V', 'VOC', 'N', 'DET', 'N', 'V', 'PRON', 'P', 'N', 'ADJ', 'LOC', 'PRON', 'CONJ', 'LOC', 'PRON', 'SUB', 'NEG', 'V', 'RES', 'PN', 'CONJ', 'NEG', 'V', 'P', 'PRON', 'N', 'CONJ', 'NEG', 'V', 'N', 'PRON', 'N', 'N', 'P', 'N', 'PN', 'REM', 'COND', 'V', 'PRON', 'RSLT', 'V', 'PRON', 'V', 'PRON', 'P', 'ACC', 'PRON', 'N', 'VOC', 'N', 'DET', 'N', 'P', 'INTG', 'V', 'PRON', 'P', 'PN', 'CIRC', 'NEG', 'V', 'DET', 'PN', 'CONJ', 'DET', 'PN', 'RES', 'P', 'N', 'PRON', 'INTG', 'SUP', 'NEG', 'V', 'PRON', 'VOC', 'PRON', 'DEM', 'V', 'PRON', 'P', 'REL', 'P', 'PRON', 'P', 'PRON', 'N', 'REM', 'P', 'INTG', 'V', 'PRON', 'P', 'REL', 'V', 'P', 'PRON', 'P', 'PRON', 'N', 'REM', 'PN', 'V', 'CONJ', 'PRON', 'NEG', 'V', 'PRON', 'NEG', 'V', 'PN', 'PN', 'CONJ', 'NEG', 'PN', 'SUP', 'AMD', 'V', 'N', 'ADJ', 'CONJ', 'NEG', 'V', 'P', 'DET', 'N', 'ACC', 'N', 'DET', 'N', 'P', 'PN', 'EMPH', 'REL', 'V', 'PRON', 'PRON', 'CONJ', 'DEM', 'DET', 'N', 'CONJ', 'REL', 'V', 'PRON', 'REM', 'PN', 'N', 'DET', 'N', 'V', 'N', 'P', 'N', 'DET', 'N', 'SUB', 'V', 'PRON', 'PRON', 'REM', 'NEG', 'V', 'PRON', 'RES', 'N', 'PRON', 'CIRC', 'NEG', 'V', 'PRON', 'VOC', 'N', 'DET', 'N', 'P', 'INTG', 'V', 'PRON', 'P', 'N', 'PN', 'CIRC', 'PRON', 'V', 'PRON', 'VOC', 'N', 'DET', 'N', 'P', 'INTG', 'V', 'PRON', 'DET', 'N', 'P', 'DET', 'N', 'CONJ', 'V', 'PRON', 'DET', 'N', 'CIRC', 'PRON', 'V', 'PRON', 'REM', 'V', 'N', 'P', 'N', 'DET', 'N', 'V', 'PRON', 'P', 'REL', 'V', 'P', 'REL', 'V', 'PRON', 'T', 'DET', 'N', 'CONJ', 'V', 'PRON', 'T', 'PRON', 'ACC', 'PRON', 'V', 'PRON', 'CONJ', 'PRO', 'V', 'PRON', 'EXP', 'P', 'REL', 'V', 'N', 'PRON', 'V', 'ACC', 'DET', 'N', 'N', 'PN', 'SUB', 'V', 'N', 'N', 'REL', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'PRON', 'LOC', 'N', 'PRON', 'V', 'ACC', 'DET', 'N', 'P', 'N', 'PN', 'V', 'PRON', 'REL', 'V', 'REM', 'PN', 'N', 'ADJ', 'V', 'P', 'N', 'PRON', 'REL', 'V', 'REM', 'PN', 'N', 'DET', 'N', 'DET', 'ADJ', 'REM', 'P', 'N', 'DET', 'N', 'REL', 'COND', 'V', 'PRON', 'P', 'N', 'V', 'PRON', 'P', 'PRON', 'CONJ', 'P', 'PRON', 'REL', 'COND', 'V', 'PRON', 'P', 'N', 'NEG', 'V', 'PRON', 'P', 'PRON', 'RES', 'SUB', 'V', 'PRON', 'P', 'PRON', 'N', 'DEM', 'P', 'ACC', 'PRON', 'V', 'PRON', 'V', 'P', 'PRON', 'P', 'DET', 'N', 'N', 'REM', 'V', 'PRON', 'P', 'PN', 'DET', 'N', 'CIRC', 'PRON', 'V', 'PRON', 'ANS', 'COND', 'V', 'P', 'N', 'PRON', 'CONJ', 'V', 'RSLT', 'ACC', 'PN', 'V', 'DET', 'N', 'ACC', 'REL', 'V', 'PRON', 'P', 'N', 'PN', 'CONJ', 'N', 'PRON', 'N', 'ADJ', 'DEM', 'NEG', 'N', 'P', 'PRON', 'P', 'DET', 'N', 'CONJ', 'NEG', 'V', 'PRON', 'PN', 'CONJ', 'NEG', 'V', 'P', 'PRON', 'T', 'DET', 'N', 'CONJ', 'NEG', 'V', 'PRON', 'CONJ', 'P', 'PRON', 'N', 'ADJ', 'REM', 'ACC', 'P', 'PRON', 'EMPH', 'N', 'V', 'PRON', 'N', 'PRON', 'P', 'DET', 'N', 'PRP', 'V', 'PRON', 'PRON', 'P', 'DET', 'N', 'REM', 'NEG', 'PRON', 'P', 'DET', 'N', 'CONJ', 'V', 'PRON', 'PRON', 'P', 'N', 'PN', 'REM', 'NEG', 'PRON', 'P', 'N', 'PN', 'CONJ', 'V', 'PRON', 'P', 'PN', 'DET', 'N', 'CIRC', 'PRON', 'V', 'PRON', 'NEG', 'V', 'P', 'N', 'SUB', 'V', 'PRON', 'PN', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'N', 'CONJ', 'V', 'P', 'DET', 'N', 'V', 'PRON', 'N', 'P', 'PRON', 'P', 'N', 'PN', 'REM', 'AMD', 'V', 'PRON', 'N', 'P', 'SUB', 'V', 'PRON', 'V', 'PRON', 'DET', 'N', 'CONJ', 'P', 'SUB', 'V', 'PRON', 'V', 'PRON', 'CONJ', 'NEG', 'V', 'PRON', 'SUB', 'V', 'PRON', 'DET', 'N', 'CONJ', 'DET', 'N', 'N', 'INTG', 'V', 'PRON', 'P', 'DET', 'N', 'T', 'T', 'PRON', 'N', 'CONJ', 'T', 'V', 'PN', 'N', 'DET', 'N', 'EMPH', 'REL', 'V', 'PRON', 'PRON', 'P', 'N', 'CONJ', 'N', 'CONJ', 'V', 'PRON', 'N', 'ADJ', 'P', 'REL', 'LOC', 'PRON', 'EMPH', 'V', 'EMPH', 'P', 'PRON', 'CONJ', 'EMPH', 'V', 'EMPH', 'PRON', 'V', 'INTG', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'P', 'DEM', 'N', 'PRON', 'V', 'PRON', 'V', 'PRON', 'V', 'REM', 'V', 'PRON', 'REM', 'PRON', 'LOC', 'PRON', 'P', 'DET', 'N', 'REM', 'COND', 'V', 'T', 'DEM', 'RSLT', 'DEM', 'PRON', 'DET', 'N', 'INTG', 'SUP', 'N', 'N', 'PN', 'V', 'PRON', 'CIRC', 'P', 'PRON', 'V', 'REL', 'P', 'DET', 'N', 'CONJ', 'DET', 'N', 'N', 'CONJ', 'N', 'CIRC', 'P', 'PRON', 'V', 'PRON', 'V', 'V', 'PRON', 'P', 'PN', 'CONJ', 'REL', 'V', 'P', 'PRON', 'CONJ', 'REL', 'V', 'P', 'PN', 'CONJ', 'PN', 'CONJ', 'PN', 'CONJ', 'PN', 'CONJ', 'DET', 'N', 'CONJ', 'REL', 'V', 'PN', 'CONJ', 'PN', 'CONJ', 'DET', 'N', 'P', 'N', 'PRON', 'NEG', 'V', 'LOC', 'N', 'P', 'PRON', 'REM', 'PRON', 'P', 'PRON', 'N', 'REM', 'COND', 'V', 'N', 'DET', 'PN', 'N', 'RSLT', 'NEG', 'V', 'P', 'PRON', 'REM', 'PRON', 'P', 'DET', 'N', 'P', 'DET', 'N', 'INTG', 'V', 'PN', 'N', 'V', 'PRON', 'T', 'N', 'PRON', 'CONJ', 'V', 'PRON', 'ACC', 'DET', 'N', 'N', 'CONJ', 'V', 'PRON', 'DET', 'N', 'REM', 'PN', 'NEG', 'V', 'DET', 'N', 'DET', 'ADJ', 'DEM', 'N', 'PRON', 'ACC', 'P', 'PRON', 'N', 'PN', 'CONJ', 'DET', 'N', 'CONJ', 'DET', 'N', 'N', 'N', 'P', 'PRON', 'NEG', 'V', 'P', 'PRON', 'DET', 'N', 'CONJ', 'NEG', 'PRON', 'V', 'PRON', 'EXP', 'REL', 'V', 'PRON', 'P', 'N', 'DEM', 'CONJ', 'V', 'PRON', 'REM', 'ACC', 'PN', 'N', 'ADJ', 'ACC', 'REL', 'V', 'PRON', 'T', 'N', 'PRON', 'CONJ', 'V', 'PRON', 'N', 'NEG', 'V', 'N', 'PRON', 'REM', 'DEM', 'PRON', 'DET', 'N', 'ACC', 'REL', 'V', 'PRON', 'CONJ', 'V', 'PRON', 'CIRC', 'PRON', 'N', 'REM', 'NEG', 'V', 'P', 'N', 'PRON', 'N', 'DET', 'N', 'N', 'CIRC', 'COND', 'V', 'P', 'PRON', 'DEM', 'P', 'PRON', 'N', 'ADJ', 'CONJ', 'NEG', 'P', 'PRON', 'P', 'N']]

In [ ]:
for i in range(len(patterns)):
    patterns[i] = {"pattern": patterns[i]}
                                      
patterns

In [ ]:
# insert patterns
mydb["patterns"].insert_many(patterns)

In [ ]:
x = quran.find({"SURAH_NUMBER": "67"})

Remove space in patterns string

In [ ]:
patterns = list(mydb["patterns"].find())

for pattern in patterns:
    stripped_spaces = pattern["pattern"].replace(" ", "")
    mydb["patterns"].update_one({"pattern": pattern["pattern"]}, {"$set" : {"pattern": stripped_spaces}})

## 6. Arabic Entity Classifier Using Patterns

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[14]:

import pandas as pd

def group(ayats):
        
    length = len(ayats)
    i = 0

    grouppedAyats = [[]]

    index = 0
    for row in ayats:

        # tokenizing location
        if index < length-1:
            currentLocation = row["AYAT_NUMBER"]
            nextLocation = ayats[index+1]['AYAT_NUMBER']

            if index == length-2 or currentLocation != nextLocation:
                i = i+1
                grouppedAyats.append([])
            
            ayats[index]['AYAT_NUMBER'] = i
            
            words = {
                'OPEN TAG' : '',
                'AYAT' : int(row['AYAT_NUMBER']),
                'ARAB' : row['ARAB'],
                'TAG' : row['TAG'],
                'WORD_NUMBER' : row['WORD_NUMBER'],
                'AYAT_NUMBER' : row['AYAT_NUMBER'],
                'CLOSE TAG' : ''
            }
            
            grouppedAyats[i].append(words)

        index += 1
        
    return grouppedAyats

def ungroup(ayats):

    ungrouppedAyats = []

    for ayat in ayats:
        for word in ayat:
            ungrouppedAyats.append(word)

    return ungrouppedAyats

def classify(patterns, ayats):

    # Prepare data
    ayats = group(ayats)

    # Rule based logic
    ayatIndex = 0
    entityIndex = 0

    for ayat in ayats:
        for pattern in patterns:
            patternLength = len(pattern)
            for wordIndex in range(len(ayat) - patternLength):
                # Get Array of words based on the pattern length
                wordTagsArray = []
                for wordOfArrayIndex in range(patternLength):
                    wordTagsArray.append(ayat[wordIndex+wordOfArrayIndex]['TAG'])
                if wordTagsArray == pattern:
                    print('foundd')
                    ayat[wordIndex]['OPEN TAG'] += str(entityIndex) + '('
                    ayat[wordIndex+patternLength]['CLOSE TAG'] += str(entityIndex) + ')'
                    entityIndex += 1
        ayatIndex += 1

    return ungroup(ayats)

# %%